# Решение задачи:

Установим библиотеку FastAPI и вспомогательные компоненты:

In [ ]:
!pip install fastapi #установим FastAPI
!pip install uvicorn #установим ASGI-сервер
!pip install python-multipart #необходимая зависимость для FastAPI (для работы с данными отправленных форм на сайте)

Этот код представляет собой **REST API** сервис на базе **FastAPI** для анализа тональности отзывов с использованием нейросетевой модели **Keras**. В начале кода происходит загрузка датасета **IMDb**, подготовка данных через векторизацию **bag-of-words** и нормализацию. Обучается нейросеть с регуляризацией **L2**, **dropout** и **batch normalization**, после чего модель сохраняется в файл *imdb_model.h5*.

Создано **FastAPI** приложение с расширенной метадатой: название, описание, версия, контактная информация и лицензия **Apache 2.0**. Определены два класса Pydantic: `ReviewCreate` (с текстом отзыва и заметкой) и `ReviewItem` (наследуется от `ReviewCreate`, добавляет поля `prediction` и `sentiment_score`). Для временного хранения данных используется словарь `datastore`, а для генерации уникальных ID — функция `generate_random_id`.

API предоставляет следующие эндпоинты:

**/status** — проверка состояния сервиса с метриками точности на тестовых данных;

**/classes** — информация о классах тональности (positive/negative);

**CRUD-операции** через /reviews/ (добавление, получение, обновление и удаление записей).

При создании и обновлении записей модель автоматически рассчитывает тональность отзыва и его уверенность, сохраняя результаты в `datastore`. Все эндпоинты снабжены тегами, подробными описаниями, примерами запросов/ответов и обработкой ошибок (например, **HTTPException** с кодом **404** при отсутствии записи).

Сервис возвращает данные в формате **JSON** через `JSONResponse`, используя `jsonable_encoder` для корректного преобразования объектов. Для документации автоматически генерируются **Swagger** и **ReDoc** интерфейсы с описанием параметров, статусов и примеров использования. Модель возвращает вероятность принадлежности к положительному классу, которая интерпретируется как уверенность в предсказании (например, 0.03 → 97% уверенности в негативном отзыве).

In [ ]:
%%writefile main.py

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
import secrets
import string

#параметры модели
max_words = 10000  #размер словаря
model_path = "imdb_model.h5"

#загрузка данных
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_words)

#функция векторизации
def vectorize_sequences(sequences, dimension=max_words):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

#предобработка данных
x_train = vectorize_sequences(x_train)
x_test = vectorize_sequences(x_test)
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')

#нормализация
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

#веса классов
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

#создание модели
model = Sequential([
    Dense(256, activation='relu', kernel_regularizer=l2(0.002), input_shape=(max_words,)),
    BatchNormalization(),
    Dropout(0.7),
    Dense(128, activation='relu', kernel_regularizer=l2(0.002)),
    BatchNormalization(),
    Dropout(0.6),
    Dense(64, activation='relu', kernel_regularizer=l2(0.002)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])

#компиляция модели
optimizer = Adam(learning_rate=0.0002)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

#callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr=1e-6)

#обучение модели
history = model.fit(
    x_train, y_train,
    epochs=80,
    batch_size=128,
    validation_split=0.2,
    class_weight=class_weights,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

model.save(model_path) #сохранение модели

model = load_model(model_path)#загрузка модели

#global variables для метрик
GLOBAL_X_TEST = x_test
GLOBAL_Y_TEST = y_test

#информация о классах
CLASS_INFO = {
    0: {"sentiment": "negative", "description": "Негативный отзыв"},
    1: {"sentiment": "positive", "description": "Позитивный отзыв"}
}

#FastAPI приложение
app = FastAPI(
    title="IMDb Sentiment Analysis API",
    description="REST API для анализа тональности отзывов",
    version="1.0.0",
    contact={
        "name": "Support",
        "email": "temich",
    },
    license_info={
        "name": "Apache 2.0",
        "url": "https://www.apache.org/licenses/LICENSE-2.0.html "
    }
)

#модели данных
class ReviewCreate(BaseModel):
    text: str = Field(..., example="This movie was terrible!", description="Текст отзыва")
    note: str = Field(..., example="Sample review", description="Заметка об отзыве")

class ReviewItem(ReviewCreate):
    prediction: str
    sentiment_score: float


datastore = {} #временное хранилище

def generate_random_id(length=8):
    characters = string.ascii_letters + string.digits
    return ''.join(secrets.choice(characters) for _ in range(length))


@app.get("/status",
         tags=["Service Monitoring"],
         summary="Проверка состояния сервиса",
         description="Возвращает текущий статус сервиса и информацию о модели и точности")
def get_status():
    predictions = (model.predict(GLOBAL_X_TEST) > 0.5).astype("int32")
    accuracy = np.mean(predictions.flatten() == GLOBAL_Y_TEST)
    return {
        "status": "OK",
        "model_type": "Keras Sequential",
        "classes": list(CLASS_INFO[0].values()),
        "accuracy": float(accuracy),
        "description": "FastAPI сервис для анализа тональности отзывов"
    }


@app.get("/classes",
         tags=["Classification Info"],
         summary="Получить информацию о классах",
         description="Возвращает детальную информацию о классах тональности")
def get_classes():
    return CLASS_INFO


@app.post("/reviews/",
          tags=["CRUD"],
          status_code=201,
          summary="Добавить новый отзыв",
          description="Принимает текст отзыва, сохраняет с уникальным ID и предсказанием")
def create_item(data: ReviewCreate):
    while True:
        item_id = generate_random_id()
        if item_id not in datastore:
            break

    #обработка текста
    word_index = imdb.get_word_index()
    text_sequence = [[word_index[word] + 3 if word in word_index else 2 for word in data.text.split()]]
    text_vector = vectorize_sequences(text_sequence)
    text_vector = text_vector / text_vector.max()

    #получение предсказания
    prediction = model.predict(text_vector)[0][0]
    sentiment_class = int(round(prediction))
    confidence = float(prediction) if sentiment_class == 1 else 1 - float(prediction)

    item = ReviewItem(
        **data.dict(),
        prediction=CLASS_INFO[sentiment_class]["sentiment"],
        sentiment_score=confidence
    )

    datastore[item_id] = item
    return {"id": item_id, **item.dict()}


@app.get("/reviews/",
         tags=["CRUD"],
         summary="Получить все отзывы",
         description="Возвращает список всех сохраненных отзывов")
def get_all_items():
    return {
        "count": len(datastore),
        "items": [{"id": id, **item.dict()} for id, item in datastore.items()]
    }


@app.get("/reviews/{item_id}",
         tags=["CRUD"],
         summary="Получить отзыв по ID",
         description="Возвращает информацию о конкретном отзыве")
def read_item(item_id: str):
    if item_id not in datastore:
        raise HTTPException(status_code=404, detail="Item не найден")
    return {"id": item_id, **datastore[item_id].dict()}


@app.put("/reviews/{item_id}",
         tags=["CRUD"],
         summary="Обновить отзыв",
         description="Обновляет данные отзыва по ID")
def update_item(item_id: str, data: ReviewCreate):
    if item_id not in datastore:
        raise HTTPException(status_code=404, detail="Item не найден")

    #обработка текста
    word_index = imdb.get_word_index()
    text_sequence = [[word_index[word] + 3 if word in word_index else 2 for word in data.text.split()]]
    text_vector = vectorize_sequences(text_sequence)
    text_vector = text_vector / text_vector.max()

    #получение предсказания
    prediction = model.predict(text_vector)[0][0]
    sentiment_class = int(round(prediction))
    confidence = float(prediction) if sentiment_class == 1 else 1 - float(prediction)

    updated_item = ReviewItem(
        **data.dict(),
        prediction=CLASS_INFO[sentiment_class]["sentiment"],
        sentiment_score=confidence
    )

    datastore[item_id] = updated_item
    return {"id": item_id, **updated_item.dict()}


@app.delete("/reviews/{item_id}",
            tags=["CRUD"],
            summary="Удалить отзыв",
            description="Удаляет отзыв по ID")
def delete_item(item_id: str):
    if item_id not in datastore:
        raise HTTPException(status_code=404, detail="Item не найден")
    del datastore[item_id]
    return {"message": "Item успешно удален"}

Overwriting main.py


`!pkill -f uvicorn` - команда принудительно завершает все процессы **uvicorn** (сервер **FastAPI**) на системе.

`nohup` и `&` - прописываются для запуска процесса в фоне (чтобы не блокировать **Colab**).

Параметр `--reload` позволяет автоматически перезапускать uvicorn при изменениях в файле **main.py**.

In [ ]:
!pkill -f uvicorn
!nohup uvicorn main:app --reload &

nohup: appending output to 'nohup.out'


Посмотрим содержимое файла nohup.out (логи работы фонового процесса) и убедимся в корректности работы:

In [ ]:
!cat nohup.out

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [1335] using StatReload


Ищем все запущенные процессы uvicorn (сервер FastAPI) и показывает их статус, чтобы убедиться в работе сервера:

In [ ]:
!ps aux | grep uvicorn

root        1335  9.0  0.1  35152 25512 ?        S    17:51   0:00 /usr/bin/python3 /usr/local/bin/uvicorn main:app --reload
root        1349  0.0  0.0   7376  3492 ?        S    17:51   0:00 /bin/bash -c ps aux | grep uvicorn
root        1351  0.0  0.0   6484  2356 ?        S    17:51   0:00 grep uvicorn


Дополнительно проверим, жив ли сервер (аналог "пинга") узнав, что эндпоинт `/status` возвращает ожидаемый результат:

In [ ]:
!curl -s http://localhost:8000/status | grep "OK"

Эта команда открывает публичный URL для нашего локального **FastAPI-сервера** (на localhost:8000) через сервис **localhost.run**:

In [ ]:
!ssh -o "StrictHostKeyChecking no" -R 80:localhost:8000 nokey@localhost.run


Welcome to localhost.run!

Follow your favourite reverse tunnel at [https://twitter.com/localhost_run].

To set up and manage custom domains go to https://admin.localhost.run/

More details on custom domains (and how to enable subdomains of your custom
domain) at https://localhost.run/docs/custom-domains

If you get a permission denied error check the faq for how to connect with a key or
create a free tunnel without a key at [http://localhost:3000/docs/faq#generating-an-ssh-key].

To explore using localhost.run visit the documentation site:
https://localhost.run/docs/


** your connection id is 2f841c99-cf3a-4aee-8b7f-5bf06747f95b, please mention it if you send me a message about an issue. **

authenticated as anonymous user
a15c8819ab57a3.lhr.life tunneled with tls termination, https://a15c8819ab57a3.lhr.life
create an account and add your key for a longer lasting domain name. see https://localhost.run/docs/forever-free/ for more information.
Open your tunnel address on your mobile w